### VQE（Variational quantum eigensolver）
パラメータ付き量子回路で変分的に基底状態を求めましょう。

### 必要なライブラリをインポート

In [1]:
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import Qubit,QubitBra,measure_all,measure_partial
from sympy.physics.quantum.gate import X,Y,Z,H,CNOT,SWAP,CPHASE,CGateS
from sympy.physics.quantum.gate import IdentityGate as _I
from sympy.physics.quantum.gate import UGate as U

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import scipy.optimize
import scipy.linalg
import numpy as np
import sys

### 変分量子回路で利用するためのゲートを定義

In [3]:
def Rxi(n,t): return U(n,represent(cos(t)*_I(1)*_I(0)-I*sin(t)*X(1)*_I(0),nqubits=2))
def Rix(n,t): return U(n,represent(cos(t)*_I(1)*_I(0)-I*sin(t)*_I(1)*X(0),nqubits=2))
def Rzi(n,t): return U(n,represent(cos(t)*_I(1)*_I(0)-I*sin(t)*Z(1)*_I(0),nqubits=2))
def Riz(n,t): return U(n,represent(cos(t)*_I(1)*_I(0)-I*sin(t)*_I(1)*Z(0),nqubits=2))

print(Rxi((0,1),pi/4).get_target_matrix())
print(Rix((0,1),pi/4).get_target_matrix())
print(Rzi((0,1),pi/4).get_target_matrix())
print(Riz((0,1),pi/4).get_target_matrix())

Matrix([[sqrt(2)/2, 0, -sqrt(2)*I/2, 0], [0, sqrt(2)/2, 0, -sqrt(2)*I/2], [-sqrt(2)*I/2, 0, sqrt(2)/2, 0], [0, -sqrt(2)*I/2, 0, sqrt(2)/2]])
Matrix([[sqrt(2)/2, -sqrt(2)*I/2, 0, 0], [-sqrt(2)*I/2, sqrt(2)/2, 0, 0], [0, 0, sqrt(2)/2, -sqrt(2)*I/2], [0, 0, -sqrt(2)*I/2, sqrt(2)/2]])
Matrix([[sqrt(2)/2 - sqrt(2)*I/2, 0, 0, 0], [0, sqrt(2)/2 - sqrt(2)*I/2, 0, 0], [0, 0, sqrt(2)/2 + sqrt(2)*I/2, 0], [0, 0, 0, sqrt(2)/2 + sqrt(2)*I/2]])
Matrix([[sqrt(2)/2 - sqrt(2)*I/2, 0, 0, 0], [0, sqrt(2)/2 + sqrt(2)*I/2, 0, 0], [0, 0, sqrt(2)/2 - sqrt(2)*I/2, 0], [0, 0, 0, sqrt(2)/2 + sqrt(2)*I/2]])


### ハミルトニアンの定義

$ H = \frac{1}{2} \left( S_{ii} \mathbb{1} + S_{ix} \sigma_{x}^{1} + S_{iz} \sigma_{z}^{1} + S_{xi} \sigma_{x}^{0}  + S_{zi} \sigma_{z}^{0} 
+ S_{xx} \sigma_{x}^{0} \sigma_{x}^{1} + S_{xz} \sigma_{x}^{0} \sigma_{z}^{1} + S_{zx} \sigma_{z}^{0} \sigma_{x}^{1}
+ S_{zz} \sigma_{z}^{0} \sigma_{z}^{1} \right) $

In [ ]:
Sii,Six,Sxi,Sxx,Szz,Siz,Szi,Sxz,Szx = symbols('Sii Six Sxi Sxx Szz Siz Szi Sxz Szx')
# 【設問2-1】定義にあるハミルトニアンを完成してください。
Hamiltonian = (   Sii *_I(0) *_I(1) 
                + Six *_I(0) * X(1)
                + Siz *_I(0) * Z(1)
                + Sxi * X(0) *_I(1)
                + Szi * Z(0) *_I(1)
                + Sxx # ***回答欄*** #
                + Sxz # ***回答欄*** #
                + Szx # ***回答欄*** # 
                + Szz * Z(0) * Z(1) )/2
h = represent(Hamiltonian,nqubits=2)
Hamiltonian

$ HeH^+ $ 分子のハミルトニアンの数値設定（パラメータはNature Communication 2014 より)

In [5]:
H_valued = h.subs([
    (Sii,-3.8505),
    (Six,-0.2288),
    (Sxi,-0.2288),
    (Siz,-1.0466),
    (Szi,-1.0466),
    (Sxx, 0.2613),
    (Sxz, 0.2288),
    (Szx, 0.2288),
    (Szi,-1.0466),
    (Szz,0.2356)])
H_valued

Matrix([
[-2.85405,        0,        0,  0.13065],
[       0, -2.04305,  0.13065,  -0.2288],
[       0,  0.13065, -2.04305,  -0.2288],
[ 0.13065,  -0.2288,  -0.2288, -0.76085]])

### 答えを事前に計算
このような小さいサイズのMatrixであれば、厳密対角化も計算で、固有値は求められます。

In [20]:
# sympy は数値計算が苦手なので対角化は時間がかかります.
# ↓ sympy で提供されている対角化
# P, D = H_valued.diagonalize() 
# ↓ sympy で提供されている固有値, 固有ベクトルの求め方
E = H_valued.eigenvects()
M = np.argmin([re(E[i][0]) for i in range(len(E))])
pprint(E[M])
print(re(E[M][0]))

⎛                                 ⎡⎡-15.2436817570853 - 7.50558636857843e-28⋅ⅈ
⎜                                 ⎢⎢                                          
⎜                                 ⎢⎢0.240786150597669 + 1.14522305828383e-30⋅ⅈ
⎜-2.86262076407668 - 0.e-22⋅ⅈ, 1, ⎢⎢                                          
⎜                                 ⎢⎢0.240786150597669 - 1.97436109465294e-26⋅ⅈ
⎜                                 ⎢⎢                                          
⎝                                 ⎣⎣                   1.0                    

⎤⎤⎞
⎥⎥⎟
⎥⎥⎟
⎥⎥⎟
⎥⎥⎟
⎥⎥⎟
⎦⎦⎠
-2.86262076407668


In [21]:
# ↓数値計算は, 固有値を求めるのが得意な numpy でも試します. 
l, p = np.linalg.eig( np.array( H_valued.tolist(), dtype=np.complex128 ))
v = np.transpose(p)
mini = np.argmin(l)
E_answer = l[mini]
#print(p)
#print(l)
print(np.array([v[mini]]))
print(E_answer)

[[ 0.99760736+0.j -0.01575801+0.j -0.01575801+0.j -0.06544399+0.j]]
(-2.8626207640766825+0j)


In [8]:
def dice(n): return [np.random.rand() for i in range(n)]
def vqe_trial(phi):
    global count
    global f
    count += 1
    trial_func = Rxi((0,1), phi[0]).get_target_matrix() \
        * Riz((0,1), phi[1]).get_target_matrix() \
        * represent(CNOT(0,1),nqubits=2) \
        * Riz((0,1), phi[2]).get_target_matrix() \
        * Rix((0,1), phi[3]).get_target_matrix() \
        * Rzi((0,1), phi[4]).get_target_matrix() \
        * Rxi((0,1), phi[5]).get_target_matrix()
    # 【設問2-2】試行関数マトリックス（trial_func）の複素共役を設定してください。
    trial_func_dag = # ***回答欄*** #
    trial = trial_func_dag * H_valued * trial_func
    pr = -1*abs(((qapply(trial).tolist())[0])[0])   # *(-1) をつけなければなりません
    # print(pr)
    f.write(str(count)+ ' ' +str(pr)+ ' ' +str(pr/E_answer)+ '\n')
    f.flush()
    return pr

In [9]:
count = 0
f = sys.stdout
# 【設問2-3】上記で定義した vqe_trial を試してみましょう。何回か呼び出して結果を表示してください。
# ***回答欄*** #

1 -2.00938731271756 0.701939753226682
2 -2.26243027367454 0.790335311636807
3 -1.71447257643126 0.598917117470241
4 -1.39939629922598 0.488851445775544
5 -2.24611667062403 0.784636476759611
6 -2.07523617734658 0.724942752944761
7 -1.95482185353686 0.682878388247623
8 -2.20604916952604 0.770639686964467
9 -1.83745878938148 0.641879920819389
10 -1.94575607616102 0.679711438056522


In [ ]:
count = 0
f = open('result_VQE.txt', 'w')
# 【設問2-4】scipy.optimize.minimize() 内で vqe_trial を使って、最低エネルギー固有値を探索してください。
res = # ***回答欄*** #
f.close()

In [ ]:
# 結果を表示します。
# pprint(res)
print(res["fun"])

In [ ]:
# 【設問2-5】結果ファイルをグラフで表示してみましょう。
dat = np.loadtxt("result_VQE.txt")
# 回答欄 #